In [1]:
import numpy as np
import imutils
import time
import cv2
import os
from playsound import playsound
import threading

In [2]:
As = 0

In [3]:
corner = 0;

# 1 = Right
# 2 = Left

In [4]:
def turn_right():
    playsound('turn_right.mpeg')

In [5]:
def turn_left():
    playsound('turn_left.mpeg')

In [6]:
def loop():
    while(not As):
        if corner == 0:
            pass
        if corner == 1:
            turn_right()
            time.sleep(3)
        if corner == 2:
            turn_left()
            time.sleep(3)

In [7]:
t1 = threading.Thread(target=loop, args=())

In [8]:
t1.start()

In [9]:
class Det:
    def __init__(self, b, conf, classid):
        self.box_ = b
        self.confidence_ = conf
        self.classID_ = classid

In [10]:
blue = [255,0,0]
red = [0,0,255]
white = [255,255,255]

In [11]:
yolo = 'yolo_tiny'
confidence_thresh = 0.6
threshold = 0.3

In [12]:
labelsPath = yolo+'\\yolov3.txt'
LABELS = open(labelsPath).read().strip().split("\n")

In [13]:
np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),dtype="uint8")

In [14]:
configPath = os.path.sep.join([yolo, "yolov4.cfg"])
weightsPath = os.path.sep.join([yolo, "yolov4.weights"])


In [15]:
# print(LABELS)
# print(len(LABELS))

In [16]:
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

In [17]:
net.getUnconnectedOutLayers()

array([327, 353, 379])

In [18]:
ln = net.getLayerNames()
ln = [ln[i - 1] for i in net.getUnconnectedOutLayers()]

In [19]:
#initialization
start = 0
end = 0
fps = 0

In [20]:
#cap = cv2.VideoCapture(0)

In [21]:
cap = cv2.VideoCapture("multiple_obj.mp4")

In [22]:
(W, H) = (None, None)

In [23]:
while True:
    capture,frame = cap.read()
    #frame = cv2.rotate(frame, cv2.ROTATE_180)
    if capture:
        
        if end - start != 0:
            fps = 1/(end - start)
        start = time.time()
        
        if W is None or H is None:
            (H, W) = frame.shape[:2]
        
        blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)
        net.setInput(blob)
        layerOutputs = net.forward(ln)
        
        # initialize our lists of detected bounding boxes, confidences, and class IDs, respectively
        boxes = []
        confidences = []
        classIDs = []
        dets = []
        
        # loop over each of the layer outputs
        for output in layerOutputs:
            # loop over each of the detections
            for detection in output:
                # extract the class ID and confidence (i.e., probability) of the current object detection
                scores = detection[5:]
                classID = np.argmax(scores)
                confidence = scores[classID]
                # filter out weak predictions by ensuring the detected probability is greater than the minimum probability
                if confidence > confidence_thresh:
                    # scale the bounding box coordinates back relative to
                    # the size of the image, keeping in mind that YOLO
                    # actually returns the center (x, y)-coordinates of
                    # the bounding box followed by the boxes' width and
                    # height
                    box = detection[0:4] * np.array([W, H, W, H])
                    (centerX, centerY, width, height) = box.astype("int")
                    # use the center (x, y)-coordinates to derive the top and and left corner of the bounding box
                    x = int(centerX - (width / 2))
                    y = int(centerY - (height / 2))
                    # update our list of bounding box coordinates, confidences, and class IDs
                    #boxes.append([x, y, int(width), int(height)])
                    #boxes.sort(reverse=True, key = lambda a:a[-1]*a[-2])
                    #confidences.append(float(confidence))
                    #classIDs.append(classID)
                    new_det = Det([x, y, int(width), int(height)], confidence, classID) 
                    dets.append(new_det)
                    dets.sort(reverse=True, key = lambda a:a.box_[-1]*a.box_[-2])
                    for det in dets:
                        boxes.append(det.box_)
                        confidences.append(float(det.confidence_))
                        classIDs.append(det.classID_)
                    
        # apply non-maxima suppression to suppress weak, overlapping bounding boxes
        idxs = cv2.dnn.NMSBoxes(boxes, confidences, confidence_thresh, threshold)
        # ensure at least one detection exists
        if len(idxs) > 0:
            # loop over the indexes we are keeping
            for i in idxs.flatten():
                # extract the bounding box coordinates
                (x, y) = (boxes[i][0], boxes[i][1])
                (w, h) = (boxes[i][2], boxes[i][3])
                # draw a bounding box rectangle and label on the frame
                color = [int(c) for c in COLORS[classIDs[i]]]
                cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                text = "{}: {:.4f}".format(LABELS[classIDs[i]],
                    confidences[i])
                cv2.putText(frame, text, (x, y - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                cv2.putText(frame, "%.2f fps" % fps, (20,H//2), cv2.FONT_HERSHEY_SIMPLEX, 0.5, white, 2)
                if x + w/2 <= W/2:
                    cv2.putText(frame, "Turn Right", (W-100, 20),cv2.FONT_HERSHEY_SIMPLEX, 0.5, red, 2)
                    side = 1
                else:
                    cv2.putText(frame, "Turn Left", (20, 20),cv2.FONT_HERSHEY_SIMPLEX, 0.5, red, 2)
                    side = 2
                break # to only have bounding box corresponding to maximum confidence
        
        cv2.imshow("frame",frame)
        end = time.time()
        
        #to exit while loop when esc is pressed
        k = cv2.waitKey(5) & 0xFF
        if k == 27:
            break
    
    else:
        break

In [24]:
cap.release()
cv2.destroyAllWindows()

In [25]:
As = 1